## Cross-validation for number of hidden neuron and hidden layers -- Two Layers

In [15]:
import sys
sys.path.append("./src") # append to system path

from sklearn import cross_validation
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.patches import Rectangle
style.use('ggplot')

In [16]:
def load_lcia_data(descs_p, target_p):
    X = pd.read_csv(descs_p,header=0,index_col=None)
    X = X.fillna(0)
    y = pd.read_csv(target_p,header=0,index_col=None)
    return X.values,y.values

def mre(true_y,pred_y):
    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((true_y - pred_y) / true_y)) * 100

### Helper functions

In [35]:
def fit_descs(trn_X,val_X):
    #fit descriptors using standard scaler and PCA
    this_scaler = StandardScaler()
    pca = PCA(n_components = 40)
    
    trn_X = this_scaler.fit_transform(trn_X)
    trn_X = pca.fit_transform(trn_X)
    
    val_X = this_scaler.transform(val_X)
    val_X = pca.transform(val_X)
    return trn_X,val_X

def init_weights(shape):
    weights = tf.random_normal(shape,stddev = 0.1)
    return tf.Variable(weights)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def build_model_and_train(trn_X, trn_y, val_X,val_y,num_epoch,num_hidden_neuron, lr=0.01,beta=0.01,verbose=True):
    # high level function to create NN model
    
    #Input and Output Dim
    num_descs = trn_X.shape[1]
    num_target = trn_y.shape[1]
    
    #Placeholders
    X = tf.placeholder(tf.float32,shape=[None,num_descs])
    y = tf.placeholder(tf.float32,shape=[None,num_target])
    
    # First layer
    w1 = init_weights((num_descs,num_hidden_neuron)) 
    b1 = bias_variable([num_hidden_neuron])
    l1 = tf.add(tf.matmul(X,w1),b1)
    l1 = tf.nn.sigmoid(l1)
    
    # Second layer
    w2 = init_weights((num_hidden_neuron,num_hidden_neuron)) 
    b2 = bias_variable([num_hidden_neuron])
    l2 = tf.add(tf.matmul(l1,w2),b2)
    l2 = tf.nn.sigmoid(l2)
    
    # Output Layer
    w_out = init_weights((num_hidden_neuron,num_target))
    b_out = bias_variable([num_target])
    l_out = tf.matmul(l2,w_out) + b_out #no nonlinarity
    
    #Prediction
    pred = l_out

    #Define Cost Function 
    regularizers = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w_out)
    cost = tf.reduce_mean(tf.square(pred - y) + beta*regularizers)

    #Gridient Descent Optimizer
    optimizer = tf.train.AdagradOptimizer(learning_rate = lr).minimize(cost)

    #init session
    init = tf.global_variables_initializer()
    costs = []
    
    # Start training
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epoch):
            for i in range(0, len(trn_X),1):
                _, c = sess.run([optimizer,cost], feed_dict={X:trn_X[i:i+1], y:trn_y[i:i+1]})

            trn_score = r2_score(trn_y,sess.run(pred, feed_dict={X:trn_X, y:trn_y}))
            val_score = r2_score(val_y,sess.run(pred, feed_dict={X:val_X, y:val_y}))
            val_mre = mre(val_y,sess.run(pred,feed_dict={X:val_X,y:val_y}))

            costs.append(val_score)
            if epoch % 5 == 0 and verbose==True:
                print("Epoch = %d,Cost = %.2f,Training Accuracy = %.2f, Validation Accuracy = %.2f, Validation MRE =%.2f" % (epoch + 1,c,trn_score,val_score,val_mre))

        # final pred on the validation set
        final_pred_val = sess.run(pred,feed_dict={X:val_X})
        
    return final_pred_val


### Two Layers for CED 

### Load Training data
use Kfold to create cross-validation dataset

In [32]:
descs_p = '../data/descs/train/descs_Mar08_3839_train.csv'
target_p = '../data/target/train/CED_train.csv'
X_raw,y_raw = load_lcia_data(descs_p, target_p)

kf = KFold(n_splits=5,random_state=3)
print kf

KFold(n_splits=5, random_state=3, shuffle=False)


In [33]:
# use this to spilt CV dataset
count = 0
all_r2 = []
for trn_idx,val_idx in kf.split(X_raw,y_raw):
    count += 1
    trn_X = X_raw[trn_idx]
    val_X = X_raw[val_idx]

    trn_y = y_raw[trn_idx]
    val_y = y_raw[val_idx]
    
    #Fit data
    trn_X,val_X = fit_descs(trn_X,val_X)
    
    print "Training on Fold ",count
    this_pred = build_model_and_train(trn_X,trn_y,val_X,val_y,
                   num_epoch=800,
                   num_hidden_neuron=128,
                   lr=0.01,
                   beta=0.01,
                   verbose=True)
    this_r2 = r2_score(val_y, this_pred)
    print this_r2
    all_r2.append(this_r2)

Training on Fold  1
Epoch = 1,Cost = 1734.11,Training Accuracy = -2.73, Validation Accuracy = -5.59, Validation MRE =70.86
Epoch = 6,Cost = 226.34,Training Accuracy = -0.95, Validation Accuracy = -2.10, Validation MRE =45.79
Epoch = 11,Cost = 13.72,Training Accuracy = -0.46, Validation Accuracy = -1.11, Validation MRE =37.73
Epoch = 16,Cost = 17.70,Training Accuracy = -0.24, Validation Accuracy = -0.64, Validation MRE =34.61
Epoch = 21,Cost = 79.85,Training Accuracy = -0.08, Validation Accuracy = -0.40, Validation MRE =33.43
Epoch = 26,Cost = 176.23,Training Accuracy = 0.05, Validation Accuracy = -0.22, Validation MRE =32.10
Epoch = 31,Cost = 283.58,Training Accuracy = 0.13, Validation Accuracy = -0.11, Validation MRE =31.35
Epoch = 36,Cost = 394.53,Training Accuracy = 0.19, Validation Accuracy = -0.02, Validation MRE =30.70
Epoch = 41,Cost = 502.15,Training Accuracy = 0.24, Validation Accuracy = 0.04, Validation MRE =30.16
Epoch = 46,Cost = 601.40,Training Accuracy = 0.27, Validation 

KeyboardInterrupt: 

In [28]:
print all_r2
print np.mean(all_r2)

[-0.062014633859870605, 0.35610534984985687, -0.37294746044563576, -0.055243701255961897, 0.12546932176224512]
-0.00172622478987


### For the Acidification Model

In [34]:
descs_p = '../data/descs/train/descs_Mar08_3839_train.csv'
target_p = '../data/target/train/acidification_train.csv'
X_raw,y_raw = load_lcia_data(descs_p, target_p)

kf = KFold(n_splits=5,random_state=3)
print kf

KFold(n_splits=5, random_state=3, shuffle=False)


In [36]:
# use this to spilt CV dataset
count = 0
all_r2 = []
for trn_idx,val_idx in kf.split(X_raw,y_raw):
    count += 1
    trn_X = X_raw[trn_idx]
    val_X = X_raw[val_idx]

    trn_y = y_raw[trn_idx]
    val_y = y_raw[val_idx]
    
    #Fit data
    trn_X,val_X = fit_descs(trn_X,val_X)
    
    print "Training on Fold ",count
    this_pred = build_model_and_train(trn_X,trn_y,val_X,val_y,
                   num_epoch=800,
                   num_hidden_neuron=128,
                   lr=0.01,
                   beta=0.01,
                   verbose=True)
    this_r2 = r2_score(val_y, this_pred)
    print this_r2
    all_r2.append(this_r2)

Training on Fold  1
Epoch = 1,Cost = 2.66,Training Accuracy = 0.03, Validation Accuracy = -0.27, Validation MRE =155.28
Epoch = 6,Cost = 1.24,Training Accuracy = 0.50, Validation Accuracy = 0.06, Validation MRE =124.30
Epoch = 11,Cost = 0.87,Training Accuracy = 0.70, Validation Accuracy = 0.19, Validation MRE =109.31
Epoch = 16,Cost = 0.68,Training Accuracy = 0.78, Validation Accuracy = 0.22, Validation MRE =103.60
Epoch = 21,Cost = 0.58,Training Accuracy = 0.82, Validation Accuracy = 0.22, Validation MRE =101.13
Epoch = 26,Cost = 0.51,Training Accuracy = 0.84, Validation Accuracy = 0.22, Validation MRE =99.72
Epoch = 31,Cost = 0.45,Training Accuracy = 0.86, Validation Accuracy = 0.21, Validation MRE =98.88
Epoch = 36,Cost = 0.41,Training Accuracy = 0.87, Validation Accuracy = 0.20, Validation MRE =98.30
Epoch = 41,Cost = 0.38,Training Accuracy = 0.88, Validation Accuracy = 0.20, Validation MRE =98.19
Epoch = 46,Cost = 0.36,Training Accuracy = 0.89, Validation Accuracy = 0.19, Validati